
https://cloud.google.com/use-cases/ocr?hl=en#ocr-optical-character-recognition-with-world-class-google-cloud-ai 

In [1]:
%%capture --no-stderr
%pip install -U -q google-genai

In [2]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

https://ai.google.dev/gemini-api/docs/document-processing?lang=python

In [4]:
from google import genai
from google.genai import types
import httpx

client = genai.Client()

doc_url = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"  # Replace with the actual URL of your PDF

# Retrieve and encode the PDF byte
doc_data = httpx.get(doc_url).content

# prompt = "Summarize this document"
# prompt = "Extract abstract of this document"
prompt = "summarize the results in figure 1b"
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=doc_data,
        mime_type='application/pdf',
      ),
      prompt])
print(response.text)

Figure 1b shows AlphaFold's TM-score compared to other groups for six new protein folds identified by CASP13 assessors.  AlphaFold achieves significantly higher TM-scores than other groups for each of these folds, demonstrating its ability to predict novel protein structures with high accuracy.  One fold (T1017s2-D1) is not shown due to unavailability for publication.


In [6]:
prompt = "summarize the content in figure 1c print out all precision scores from AlphaFold"
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=doc_data,
        mime_type='application/pdf',
      ),
      prompt])
print(response.text)

Figure 1c shows the contact precisions for AlphaFold (AF) and two other top-performing methods (498 and 032) in CASP13.  The precisions are given for different lengths of contact predictions (L long, L/2 long, L/5 long), where L is the length of the protein domain.  Here are the precision scores for AlphaFold:

* **L long:** 70.1
* **L/2 long:** 85.3
* **L/5 long:** 90.6


In [10]:
doc_url = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"  # Replace with the actual URL of your PDF

# Retrieve and encode the PDF byte
doc_data = httpx.get(doc_url).content

prompt = "Summarize this document"
response = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[
      types.Part.from_bytes(
        data=doc_data,
        mime_type='application/pdf',
      ),
      prompt])
print(response.text)

This document describes AlphaFold, a new system for protein structure prediction that uses deep learning to improve accuracy. The system trains a neural network to predict the distances between pairs of residues in a protein, which is then used to construct a potential of mean force that can accurately describe the shape of a protein. The resulting system has been shown to achieve high accuracy, even for sequences with relatively few homologous sequences. In the most recent Critical Assessment of Protein Structure Prediction (CASP13), AlphaFold created high-accuracy structures for 24 out of 43 free modelling domains, whereas the next best method achieved such accuracy for only 14 out of 43 domains. The document also includes details on the methodology, data, code availability, and results of the system.

